# Experiment Tracking with Bailo

In addition to core concepts covered in previous notebooks, Bailo also offers integrations that might be useful within the wider machine learning lifecycle. This example notebook will run through **experiment tracking** in particular. The following concepts will be covered:

* Creating a new experiment using a Bailo model.
* Conducting experiment runs and logging parameters/metrics.
* Importing existing experiments from dedicated experiment tracking tools such as MLFlow Tracking or Hyperopt.
* Publishing results to the Bailo service.

Prerequisites:
* Completion of the basic notebooks, in particular **models_and_releases_demo.ipynb**.
* Python 3.8.1 or higher (including a notebook environment for this demo).
* A local or remote Bailo service (see https://github.com/gchq/Bailo).
* *Optional*: Dependencies for certain tools (e.g. MLFlow).
* *Optional*: A local or remote MLFlow server, if following the MLFlow steps.


## Introduction

In order to create an `Experiment()` object, you'll first need to have a Bailo `Model()` object, and thus a defined `Client()` too. We learned how to do this in a previous notebook, but it'll be done again in this section.

In [ ]:
# Necessary import statements

from bailo import Model, Client, Experiment

# Instantiating the Bailo client

client = Client("http://127.0.0.1:8080") # <- INSERT BAILO URL (if not hosting locally)

#model = Model.create(client=client, name="YOLOv5", description="YOLOv5 model for object detection.", team_id="uncategorised")

## Creating a new experiment

In [ ]:
model = Model.from_id(client=client, model_id="experiment-tracking-v3-cbdpg0")
experiment = model.create_experiment()

## Conducting experiment runs

In [ ]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

metrics = {
    "accuracy": 0.98,
}

for x in range(5):
    experiment.start_run()
    experiment.log_params(params)
    experiment.log_metrics(metrics)

In [ ]:
experiment.raw
experiment.publish(mc_loc="performance.performanceMetrics", run_id=0)

## Importing existing experiments from other tools

In [ ]:
import mlflow

mlflow.set_tracking_uri(uri="http://127.0.0.1:8082")
mlflow.set_experiment("Demonstator")

with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("accuracy", 0.86)
    mlflow.log_metric("recall", 0.71)
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

In [ ]:

experiment_mlflow = model.create_experiment()
experiment_mlflow.from_mlflow(tracking_uri="http://127.0.0.1:8082", experiment_id="951999943736003442")

experiment_mlflow.raw

## Publishing results to Bailo

In [ ]:
experiment_mlflow.publish(mc_loc="performance.performanceMetrics", run_id=0)